# $\epsilon-greedy$ Algorithm

$\epsilon-greedy$ is a popular exploration approach in which the agent for most of the time (with the probability of $1-\epsilon$) exploit the current greedy action, and sometimes randomly explore an action among all available actions with the probability of $\epsilon$.

## 0. Imports

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

## 1. Defining a Marketing Scenario

In this section, we want to define a hypothetical marketing scenario in which we have 5 ads each of which following a Bernoulli distribution. 

The goal is to find what is the best ad to show to the user.

But before that, since each ad follows a Bernoulli distribution, let's first implement the Bernoulli distribution.

### 1.1. Defining Bernoulli Distribution

In [2]:
class Bernoulli:
    def __init__(self, p):
        """
        Define a Bernoulli distribution

        Parameters
        ----------
        p : a float number between 0 to 1
            p represents the probability of choosing 1

        Returns
        -------
        None.

        """
        self.p = p
        
    def draw(self):
        """
        Draw a single sample from the Bernoulli distribution

        Returns
        -------
        reward : binary: 0 or 1
            A sample from the distribution
        """
        reward = np.random.binomial(n=1, p=self.p)
        return reward

### 1.2. Defining $\epsilon-greedy$ Algorithm Class

In [3]:
class EpsilonGreedy:
    def __init__(self, options, epsilon):
        """
        Parameters
        ----------
        options : list
            a list of distribution for each option. each option leads to a 
            probabilistic reward with unknown underlying distributions.
        epsilon : float
            exploration rate of the e-greedy algorithm

        Returns
        -------
        None.

        """
        self.options = options
        self.n_options = len(options)
        self.epsilon = epsilon

        self._reset()
        self._build_model()
        
        
    def _reset(self):
        """
        Define some variables to keep track of the reward and timestep.

        Returns
        -------
        None.

        """
        self.rewards = []
        self.total_reward = 0
        self.avg_reward = 0
        self.avg_rewards = []
        self.time_step = 0
        
        
    def _build_model(self):
        """
        Build a tabular model to keep track of the action values and the
        number of time each action has been selected.

        Returns
        -------
        None.

        """
        self.Q = np.zeros(self.n_options)
        self.N = np.zeros(self.n_options)
        
        
    def _get_action(self):
        """
        Select an action according to e-greedy edxploration strategy

        Returns
        -------
        action : int
            selected action to be executed in the env.

        """
        if np.random.uniform() <= self.epsilon:
            # explore non-greedy actions
            action = np.random.randint(self.n_options)
        else:
            # exploit the greedy action
            action = np.argmax(self.Q)
        return action
    
    
    def _fit(self, action, rew):
        """
        Based on the latest action and reward, we update the model

        Parameters
        ----------
        action : int
            the selected action in the current timestep.
        rew : float
            the reward the env gives to the agent accordoing to it's selected action.

        Returns
        -------
        None.

        """
        self.N[action] += 1
        self.Q[action] += (1/self.N[action]) * (rew - self.Q[action])
        
        
    def _update(self, rew):
        """
        Updating the reward related variables and time_step in each timestep.

        Parameters
        ----------
        rew : float
            the reward the env gives to the agent accordoing to it's selected action.

        Returns
        -------
        None.

        """
        self.rewards.append(rew)
        self.total_reward += rew
        self.avg_reward += (rew - self.avg_reward)/(self.time_step+1)
        self.avg_rewards.append(self.avg_reward)
        self.time_step += 1
        
        
    def _step(self, action):
        """
        Excuting the action the agent selected to see how rewarding it is.

        Parameters
        ----------
        action : int
        the selected action of the current timestep

        Returns
        -------
        None.

        """
        rew = self.options[action].draw()
        self._fit(action, rew)
        self._update(rew)
    
    
    def play(self, n_iters):
        """
        Running the algorithm for some iterations to see its performance

        Parameters
        ----------
        n_iters : int
            number of iterations.

        Returns
        -------
        None.

        """
        for i in range(n_iters):
            action = self._get_action()
            self._step(action)
        self.best_action = np.argmax(self.Q)
        
            
    def render(self):
        """
        Printing the results.

        Returns
        -------
        None.

        """
        print(f"----- e_greedy total_reward: {self.total_reward}")
        print(f"----- e_greedy avg_reward: {self.avg_reward}")
        print(f"----- e_greedy action_value: {self.Q}")
        print(f"----- e_greedy n_visits_per_ad: {self.N}")
        print(f"----- e_greedy best_action: {self.best_action}")

## 2. Find the best add by $\epsilon-greedy$ approach

In [6]:
## Define some ads following Bernoulli distribution/

b_ad_A = Bernoulli(0.03)
b_ad_B = Bernoulli(0.06)
b_ad_C = Bernoulli(0.073)
b_ad_D = Bernoulli(0.036)
b_ad_E = Bernoulli(0.027)

b_ads = [b_ad_A, b_ad_B, b_ad_C, b_ad_D, b_ad_E]

In [7]:
n_iters = 100_000

In [9]:
e_greedy = EpsilonGreedy(b_ads, epsilon=0.1)
e_greedy.play(n_iters=n_iters)
e_greedy.render()

----- e_greedy total_reward: 6827
----- e_greedy avg_reward: 0.06827000000000089
----- e_greedy action_value: [0.03090172 0.06486165 0.07078318 0.03796204 0.02772277]
----- e_greedy n_visits_per_ad: [ 1974.  3361. 90643.  2002.  2020.]
----- e_greedy best_action: 2
